# Tucker Tensors

```
Copyright 2022 National Technology & Engineering Solutions of Sandia,
LLC (NTESS). Under the terms of Contract DE-NA0003525 with NTESS, the
U.S. Government retains certain rights in this software.
```

Tucker format is a decomposition of a tensor X as the product of a core tensor G and matrices (e.g., A,B,C) in each dimension. In other words, a tensor X is expressed as:

$
\mathcal{X} = \mathcal{G} \times_1 A \times_2 B \times_3 C
$

In MATLAB notation, `X=ttm(G,{A,B,C})`.

## Contents

* [Creating a ttensor with a tensor core](#Creating-a-ttensor-with-a-tensor-core)
* [Alternate core formats: sptensor, ktensor, or ttensor](#Alternate-core-formats:-sptensor,-ktensor,-or-ttensor)
* [BREAK](#BREAK)
* [Specifying the accumulation method for the constructor](#Specifying-the-accumulation-method-for-the-constructor)
* [Creating a one-dimensional Sptensor](#Creating-a-one-dimensional-Sptensor)
* [Creating an all-zero Sptensor](#Creating-an-all-zero-Sptensor)
* [Constituent parts of a Sptensor](#Constituent-parts-of-a-Sptensor)
* [Creating a sparse tensor from its constituent parts](#Creating-a-sparse-tensor-from-its-constituent-parts)
* [Creating an empty Sptensor](#Creating-an-empty-Sptensor)
* [Use sptenrand to create a random Sptensor](#Use-sptenrand-to-create-a-random-Sptensor)
* [Use squeeze to remove singleton dimensions from a Sptensor](#Use-squeeze-to-remove-singleton-dimensions-from-a-Sptensor)
* [Use squash to remove empty slices from a Sptensor](#Use-squash-to-remove-empty-slices-from-a-Sptensor)
* [Use full or tensor to convert a sptensor to a (dense) tensor](#Use-full-or-tensor-to-convert-a-sptensor-to-a-(dense)-tensor)
* [Use Sptensor to convert a (dense) tensor to a Sptensor](#Use-Sptensor-to-convert-a-(dense)-tensor-to-a-Sptensor)
* [Use double to convert a sptensor to a (dense) multidimensional array](#Use-double-to-convert-a-sptensor-to-a-(dense)-multidimensional-array)
* [Use find to extract nonzeros from a tensor and then create a Sptensor](#Use-find-to-extract-nonzeros-from-a-tensor-and-then-create-a-Sptensor)
* [Use ndims and size to get the size of a Sptensor](#Use-ndims-and-size-to-get-the-size-of-a-Sptensor)
* [Use nnz to get the number of nonzeroes of a Sptensor](#Use-nnz-to-get-the-number-of-nonzeroes-of-a-Sptensor)
* [Subscripted reference for a Sptensor](#Subscripted-reference-for-a-Sptensor)
* [Subscripted assignment for a Sptensor](#Subscripted-assignment-for-a-Sptensor)
* [Use elemfun to manipulate the nonzeros of a Sptensor](#Use-elemfun-to-manipulate-the-nonzeros-of-a-Sptensor)
* [Basic operations (plus, minus, times, etc.) on a Sptensor](#Basic-operations-(plus,-minus,-times,-etc.)-on-a-Sptensor)
     * [Addition](#Addition)
     * [Subtraction](#Subtraction)
     * [Multiplication](#Multiplication)
     * [Division](#Division)
     * [Power](#Power)
     * [Equality](#Equality)
* [Use permute to reorder the modes of a Sptensor](#Use-permute-to-reorder-the-modes-of-a-Sptensor)
* [Displaying a Sptensor](#Displaying-a-Sptensor)

In [2]:
import pyttb as ttb
import numpy as np

## Creating a ttensor with a tensor core

In [44]:
np.random.seed(0)
core = ttb.tensor.from_data(np.random.rand(3,2,1), shape=(3,2,1)) # The core tensor.
U = [np.random.rand(5,3), np.random.rand(4,2), np.random.rand(3,1)] # The factor matrices.
X = ttb.ttensor.from_data(core, U) # Create the ttensor.
X

Tensor of shape: (5, 4, 3)
	Core is a 	tensor of shape 3 x 2 x 1
	data[0, :, :] = 
	[[0.5488135 ]
	 [0.71518937]]
	data[1, :, :] = 
	[[0.60276338]
	 [0.54488318]]
	data[2, :, :] = 
	[[0.4236548 ]
	 [0.64589411]]
	U[0] = 
		[[0.43758721 0.891773   0.96366276]
		 [0.38344152 0.79172504 0.52889492]
		 [0.56804456 0.92559664 0.07103606]
		 [0.0871293  0.0202184  0.83261985]
		 [0.77815675 0.87001215 0.97861834]]
	U[1] = 
		[[0.79915856 0.46147936]
		 [0.78052918 0.11827443]
		 [0.63992102 0.14335329]
		 [0.94466892 0.52184832]]
	U[2] = 
		[[0.41466194]
		 [0.26455561]
		 [0.77423369]]

## **TODO**: Alternate core formats: sptensor, ktensor, or ttensor

In [67]:
np.random.seed(0)
# sptensor_core = ttb.sptenrand([3,2,1], nonzeros=3) # Create a 3 x 2 x 1 sptensor.
# U = [np.random.rand(5,3), np.random.rand(4,2), np.random.rand(3,1)] # The factor matrices.
# Y = ttb.ttensor.from_data(sptensor_core, U) # # Core is a sptensor.

In [72]:
V = [np.random.rand(3,2), np.random.rand(2,2), np.random.rand(1,2)] # Create some random matrices.
# ktensor_core = ttb.ktensor.from_factor_matrices(V) # Create a 3 x 2 x 1  ktensor.
# Y = ttb.ttensor.from_data(ktensor_core, V) # Core is a ktensor.

In [103]:
tensor = ttb.tensor.from_data(np.array([2,2,2]), shape=(4,2,8))
# ttensor_core = ttb.ttensor.from_data()

AssertionError: TTB:WrongSize, Size of data does not match specified size of tensor

## BREAK

In [3]:
X = ttb.sptensor.from_data(subs,vals,(3, 5, 2))   # Or, specify the shape explicitly.
X

Sparse tensor of shape 3 x 5 x 2 with 3 nonzeros 
	[0, 0, 0] = 1
	[0, 1, 0] = 2
	[2, 3, 1] = 3

Values corresponding to repeated subscripts are summed.

In [4]:
subs = np.array([[0,0,0],[0,0,2],[2,2,2],[3,3,3],[0,0,0],[0,0,0]])  # (1,1,1) is repeated.
vals = np.array([2,2,2,2,2,2])[:, None]  # Vals is a column vector.
X = ttb.sptensor.from_aggregator(subs,vals)
X

Sparse tensor of shape 4 x 4 x 4 with 4 nonzeros 
	[0, 0, 0] = 6
	[0, 0, 2] = 2
	[2, 2, 2] = 2
	[3, 3, 3] = 2

## Specifying the accumulation method for the constructor

In [5]:
subs = np.array([[0,0,0],[0,0,2],[2,2,2],[3,3,3],[0,0,0],[0,0,0]])
vals = 2 * np.ones((6, 1))  # A column vector of 2s
shape = (4, 4, 4)
X = ttb.sptensor.from_aggregator(subs, vals, shape, np.max) # Maximum element.
X

Sparse tensor of shape 4 x 4 x 4 with 4 nonzeros 
	[0, 0, 0] = 2.0
	[0, 0, 2] = 2.0
	[2, 2, 2] = 2.0
	[3, 3, 3] = 2.0

In [6]:
myfun = myfun = lambda x: np.sum(x) / 3 # Total sum divided by three.
X = ttb.sptensor.from_aggregator(subs, vals, shape, myfun) # Custom accumulation function.
X

Sparse tensor of shape 4 x 4 x 4 with 4 nonzeros 
	[0, 0, 0] = 2.0
	[0, 0, 2] = 0.6666666666666666
	[2, 2, 2] = 0.6666666666666666
	[3, 3, 3] = 0.6666666666666666

## Creating a one-dimensional Sptensor

In [7]:
X = ttb.sptensor.from_aggregator(np.array([[0],[2],[4]]),np.ones((3,1)))
X

Sparse tensor of shape 5 with 3 nonzeros 
	[0] = 1.0
	[2] = 1.0
	[4] = 1.0

In [8]:
np.random.seed(0)
X = ttb.sptenrand((50,), nonzeros=5)
X

Sparse tensor of shape 50 with 5 nonzeros 
	[19] = 0.7917250380826646
	[21] = 0.5288949197529045
	[32] = 0.5680445610939323
	[44] = 0.925596638292661
	[48] = 0.07103605819788694

## Creating an all-zero Sptensor

In [9]:
X = ttb.sptensor()
X[9,9,9] = 0 # Creates an all-zero tensor.
X

All-zero sparse tensor of shape 10 x 10 x 10

## Constituent parts of a Sptensor

In [10]:
np.random.seed(0)
X = ttb.sptenrand([40, 30, 20], nonzeros=5) # Create data.
X.subs # Subscripts of nonzeros.

array([[15, 23, 10],
       [17, 26, 19],
       [21, 12, 12],
       [21, 21, 12],
       [22, 27,  1]])

In [11]:
X.vals # Corresponding nonzero values.

array([[0.0871293 ],
       [0.0202184 ],
       [0.83261985],
       [0.77815675],
       [0.87001215]])

In [12]:
X.shape # The shape.

[40, 30, 20]

## Creating a sparse tensor from its constituent parts

In [13]:
np.random.seed(0)
X = ttb.sptenrand([40, 30, 20], nonzeros=5) # Create data.
Y = ttb.sptensor.from_tensor_type(X)
Y

Sparse tensor of shape 40 x 30 x 20 with 5 nonzeros 
	[15, 23, 10] = 0.08712929970154071
	[17, 26, 19] = 0.02021839744032572
	[21, 12, 12] = 0.832619845547938
	[21, 21, 12] = 0.7781567509498505
	[22, 27, 1] = 0.8700121482468192

## Creating an empty Sptensor

In [14]:
Y = ttb.sptensor() # Create an empty sptensor.
Y

All-zero sparse tensor of shape ()

## Use sptenrand to create a random Sptensor

In [15]:
np.random.seed(0)
X = ttb.sptenrand([10, 10, 10], 0.01) # Create a tesnor with 1% nonzeros using the 'density' param.
X

Sparse tensor of shape 10 x 10 x 10 with 10 nonzeros 
	[0, 0, 8] = 0.26455561210462697
	[1, 6, 1] = 0.7742336894342167
	[3, 7, 5] = 0.45615033221654855
	[4, 8, 9] = 0.5684339488686485
	[5, 4, 6] = 0.018789800436355142
	[5, 7, 6] = 0.6176354970758771
	[5, 9, 0] = 0.6120957227224214
	[7, 4, 7] = 0.6169339968747569
	[7, 8, 9] = 0.9437480785146242
	[9, 5, 4] = 0.6818202991034834

In [16]:
np.random.seed(0)
X = ttb.sptenrand([10, 10, 10], nonzeros=10) # Create a tensor with 10 nonzeros using the 'nonzeros' param.
X

Sparse tensor of shape 10 x 10 x 10 with 10 nonzeros 
	[0, 0, 8] = 0.26455561210462697
	[1, 6, 1] = 0.7742336894342167
	[3, 7, 5] = 0.45615033221654855
	[4, 8, 9] = 0.5684339488686485
	[5, 4, 6] = 0.018789800436355142
	[5, 7, 6] = 0.6176354970758771
	[5, 9, 0] = 0.6120957227224214
	[7, 4, 7] = 0.6169339968747569
	[7, 8, 9] = 0.9437480785146242
	[9, 5, 4] = 0.6818202991034834

## Use squeeze to remove singleton dimensions from a Sptensor

In [17]:
indices = np.array([[0, 0, 0], [1, 0, 0]])
values = np.ones((2,))

Y = ttb.sptensor.from_aggregator(indices, values) # Create a sparse tensor.
Y

Sparse tensor of shape 2 x 1 x 1 with 2 nonzeros 
	[0, 0, 0] = 1.0
	[1, 0, 0] = 1.0

In [18]:
Y.squeeze() # Remove singleton dimensions.

Sparse tensor of shape 2 with 2 nonzeros 
	[0] = 1.0
	[1] = 1.0

## Use squash to remove empty slices from a Sptensor

In [19]:
indices = np.array([[0, 0, 0], [2, 2, 2]])
values = np.array([1,3])

Y = ttb.sptensor.from_aggregator(indices, values) # Create a sparse tensor.
Y

Sparse tensor of shape 3 x 3 x 3 with 2 nonzeros 
	[0, 0, 0] = 1
	[2, 2, 2] = 3

## TODO: SQUASH IS not implemented

In [20]:
# Y.squash()

## Use full or tensor to convert a sptensor to a (dense) tensor

In [21]:
indices = np.array([[0, 0, 0], [1, 1, 1]])
values = np.array([[1], [1]])
X = ttb.sptensor.from_aggregator(indices, values) # Create a sparse tensor.
X.full() # Convert it to a (dense) tensor.

tensor of shape 2 x 2 x 2
data[0, :, :] = 
[[1. 0.]
 [0. 0.]]
data[1, :, :] = 
[[0. 0.]
 [0. 1.]]

In [22]:
Y = ttb.tensor.from_tensor_type(X) # Same as above.
Y

tensor of shape 2 x 2 x 2
data[0, :, :] = 
[[1. 0.]
 [0. 0.]]
data[1, :, :] = 
[[0. 0.]
 [0. 1.]]

## Use Sptensor to convert a (dense) tensor to a Sptensor

In [23]:
indices = np.array([[0, 0, 0], [1, 1, 1]])
values = np.array([[1], [1]])
X = ttb.sptensor.from_aggregator(indices, values) # Create a sparse tensor.
Y = ttb.tensor.from_tensor_type(X) # Convert it to a (dense) tensor.
Z = ttb.sptensor.from_tensor_type(Y) # Convert a tensor to a sptensor.
Z

Sparse tensor of shape 2 x 2 x 2 with 2 nonzeros 
	[0, 0, 0] = 1.0
	[1, 1, 1] = 1.0

## Use double to convert a sptensor to a (dense) multidimensional array

In [24]:
indices = np.array([[0, 0, 0], [1, 1, 1]])
values = np.array([[1], [1]])
X = ttb.sptensor.from_aggregator(indices, values) # Create a sparse tensor.
Y = ttb.sptensor.double(X) # Creates numpy.ndarray
Y 

array([[[1., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 1.]]])

## Use find to extract nonzeros from a tensor and then create a Sptensor

In [25]:
np.random.seed(0)
X = ttb.tensor.from_data(np.random.rand(5,4,2)) # Create a tensor.
larger_entries = X > 0.9 # Extract subscipts of values greater than 0.9.
subs, vals = larger_entries.find() # Extract corresponding subscripts and values.
Y = ttb.sptensor.from_aggregator(subs, vals) # Create a new sptensor.
Y

Sparse tensor of shape 5 x 4 x 2 with 5 nonzeros 
	[1, 0, 0] = 1
	[1, 2, 1] = 1
	[2, 2, 0] = 1
	[3, 1, 1] = 1
	[4, 3, 0] = 1

## Use ndims and size to get the size of a Sptensor

In [26]:
X = ttb.sptensor.from_data(np.array([[1,1,1],[2,3,2],[3,4,1],[1,0,0]]),np.array([[3],[2],[1],[3]]), (4, 5, 3))
X

Sparse tensor of shape 4 x 5 x 3 with 4 nonzeros 
	[1, 1, 1] = 3
	[2, 3, 2] = 2
	[3, 4, 1] = 1
	[1, 0, 0] = 3

In [27]:
X.ndims # Number of dimensions or modes.

3

In [28]:
X.shape # Size of X.

(4, 5, 3)

In [29]:
X.shape[2] # Size of mode 3 of X.

3

## Use nnz to get the number of nonzeroes of a Sptensor

In [30]:
X = ttb.sptensor.from_data(np.array([[1,1,1],[2,3,2],[3,4,1],[1,0,0]]),np.array([[3],[2],[1],[3]]), (4, 5, 3))
X.nnz # Number of nonzeros of X.

4

## Subscripted reference for a Sptensor

In [31]:
X = ttb.sptensor.from_data(np.array([[3,3,3],[1,1,0],[1,2,1]]), np.array([[3],[5],[1]]), (4, 4, 4)) # Create a sptensor.
X

Sparse tensor of shape 4 x 4 x 4 with 3 nonzeros 
	[3, 3, 3] = 3
	[1, 1, 0] = 5
	[1, 2, 1] = 1

In [32]:
X[0, 1, 0].item() # Extract the (0,1,0) element, which is zero.

0

In [33]:
X[3,3,3].item()  # Extract the (3,3,3) element, which is non-zero.

3

In [34]:
X[0:2,1:4,:] # Extract the 2x3x4 subtensor.

Sparse tensor of shape 2 x 3 x 4 with 2 nonzeros 
	[1, 0, 0] = 5
	[1, 1, 1] = 1

## **TODO** : Use .item() to extract the nested item here? Not sure how to get extraction to work with a 2D(?) array as input, like in the MATLAB version

In [35]:
X[1,1,1]

array([[0]])

In [36]:
X[1,1,0]

array([[5]])

In [37]:
X[[0, 5]] # Same as above but with linear indices.

array([[0],
       [5]])

In [38]:
indices = np.array([[0],[2],[4]])
values = np.array([1,1,1])
X = ttb.sptensor.from_aggregator(indices, values)
X

Sparse tensor of shape 5 with 3 nonzeros 
	[0] = 1
	[2] = 1
	[4] = 1

In [39]:
X[(2,)]

array([[1]])

In [40]:
X[[2,4],] # Returns a subtensor.

Sparse tensor of shape 2 with 2 nonzeros 
	[0] = 1
	[1] = 1

## **TODO** : 'extract' seems to be error-prone.

In [41]:
# X[[2,4], 'extract'] # Same as above *but* returns an array.

## Subscripted assignment for a Sptensor

In [42]:
X = ttb.sptensor.from_data(np.array([[]]), np.array([[]]), (30, 40, 20))
X

All-zero sparse tensor of shape 30 x 40 x 20

In [43]:
X[29,39,19] = 7 # Assign a single element.
X

Sparse tensor of shape 30 x 40 x 20 with 1 nonzeros 
	[29, 39, 19] = 7.0

In [44]:
X[0,0,0], X[1,1,1] = [1,1] # Assign a list of elements.
X

Sparse tensor of shape 30 x 40 x 20 with 3 nonzeros 
	[29, 39, 19] = 7.0
	[0, 0, 0] = 1.0
	[1, 1, 1] = 1.0

## **TODO**: currently producing ValueError in attempting to assign a subtensor.

In [45]:
np.random.seed(0)
Y = ttb.sptenrand((10, 10, 10), nonzeros = 10)
#  X[10:20, 10:20, 10:20] = Y # Assign a subtensor.

In [46]:
X[30,40,20] = 4 # Grows the size of the sptensor.
X

Sparse tensor of shape 31 x 41 x 21 with 4 nonzeros 
	[29, 39, 19] = 7.0
	[0, 0, 0] = 1.0
	[1, 1, 1] = 1.0
	[30, 40, 20] = 4.0

## **TODO**: Same issue here as above

In [47]:
# X[110:120, 110:120, 110:120] = ttb.sptenrand((10, 10, 10), nonzeros = 10) # Grow more.

## **TODO**: Not sure how to use end() to extract the analgous element from the ttb example.

## Use end as the last index

In [48]:
X[X.end()-10: X.end(), X.end()-10:X.end(), X.end()-5:X.end()]
# X[108:118, 110:120, 115:120]

All-zero sparse tensor of shape 0 x 0 x 0

## Use elemfun to manipulate the nonzeros of a Sptensor

In [49]:
np.random.seed(0)
X = ttb.sptenrand((10, 10, 10), nonzeros = 3)
X

Sparse tensor of shape 10 x 10 x 10 with 3 nonzeros 
	[4, 8, 9] = 0.3834415188257777
	[5, 4, 6] = 0.7917250380826646
	[5, 7, 6] = 0.5288949197529045

In [50]:
Z = X.elemfun(lambda value: np.sqrt(value)) # Square root of every nonzero.
Z

Sparse tensor of shape 10 x 10 x 10 with 3 nonzeros 
	[4, 8, 9] = 0.6192265488702642
	[5, 4, 6] = 0.8897893223020068
	[5, 7, 6] = 0.7272516206602118

In [51]:
Z = X.elemfun(lambda value: value + 1) # Use a custom function.
Z

Sparse tensor of shape 10 x 10 x 10 with 3 nonzeros 
	[4, 8, 9] = 1.3834415188257778
	[5, 4, 6] = 1.7917250380826646
	[5, 7, 6] = 1.5288949197529045

In [52]:
Z = X.ones() # Change every nonzero to one.
Z

Sparse tensor of shape 10 x 10 x 10 with 3 nonzeros 
	[4, 8, 9] = 1.0
	[5, 4, 6] = 1.0
	[5, 7, 6] = 1.0

## Basic operations (plus, minus, times, etc.) on a Sptensor

Sptensors support plus, minus, times, divide, power, equals, and not-equals operators.  Sptensors can use their operators with another Sptensor or a scalar (with the exception of equalities which only takes Sptensors).  All mathematical operators are elementwise operations.

### Addition

In [53]:
X = ttb.sptensor.from_data(np.array([[0,0],[1,1]]),np.array([[2],[2]]),(2,2))
X

Sparse tensor of shape 2 x 2 with 2 nonzeros 
	[0, 0] = 2
	[1, 1] = 2

In [54]:
Y = ttb.sptensor.from_data(np.array([[0,0],[0,1]]),np.array([[3],[3]]),(2,2))
Y

Sparse tensor of shape 2 x 2 with 2 nonzeros 
	[0, 0] = 3
	[0, 1] = 3

In [55]:
+X # Calls uplus.

Sparse tensor of shape 2 x 2 with 2 nonzeros 
	[0, 0] = 2
	[1, 1] = 2

In [56]:
X + 1  # This addition yields dense tensor

tensor of shape 2 x 2
data[:, :] = 
[[3. 1.]
 [1. 3.]]

In [57]:
X + Y  # This addition yields sparse tensor

Sparse tensor of shape 2 x 2 with 3 nonzeros 
	[0, 0] = 5
	[0, 1] = 3
	[1, 1] = 2

In [58]:
X += 2
X

tensor of shape 2 x 2
data[:, :] = 
[[4. 2.]
 [2. 4.]]

### Subtraction

In [59]:
X = ttb.sptensor.from_data(np.array([[0,0],[1,1]]),np.array([[2],[2]]),(2,2))
Y = ttb.sptensor.from_data(np.array([[0,0],[0,1]]),np.array([[3],[3]]),(2,2))

In [60]:
-X # Calls uminus.

Sparse tensor of shape 2 x 2 with 2 nonzeros 
	[0, 0] = -2
	[1, 1] = -2

In [61]:
X - Y # Calls minus.

Sparse tensor of shape 2 x 2 with 3 nonzeros 
	[0, 0] = -1
	[0, 1] = -3
	[1, 1] = 2

### Multiplication

In [62]:
X = ttb.sptensor.from_data(np.array([[0,0],[1,1]]),np.array([[2],[2]]),(2,2))
Y = ttb.sptensor.from_data(np.array([[0,0],[0,1]]),np.array([[3],[3]]),(2,2))

In [63]:
X * Y # Calls times.

Sparse tensor of shape 2 x 2 with 1 nonzeros 
	[0, 0] = 6

In [64]:
X * 5 # Calls mtimes.

Sparse tensor of shape 2 x 2 with 2 nonzeros 
	[0, 0] = 10
	[1, 1] = 10

### Division

In [65]:
X = ttb.sptensor.from_data(np.array([[0,0],[1,1]]),np.array([[2],[2]]),(2,2))
Y = ttb.sptensor.from_data(np.array([[0,0],[0,1]]),np.array([[3],[3]]),(2,2))

In [66]:
X / 2 # Calls rdivide.

Sparse tensor of shape 2 x 2 with 2 nonzeros 
	[0, 0] = 1.0
	[1, 1] = 1.0

In [67]:
X / Y # Divide by Zero 

Sparse tensor of shape 2 x 2 with 4 nonzeros 
	[0, 0] = 0.6666666666666666
	[1, 0] = nan
	[1, 1] = 0.0
	[1, 0] = nan

In [68]:
X /= 4
print(X)

Sparse tensor of shape 2 x 2 with 2 nonzeros 
	[0, 0] = 0.5
	[1, 1] = 0.5


### Equality

In [69]:
X = ttb.sptensor.from_data(np.array([[0,0],[1,1]]),np.array([[2],[2]]),(2,2))
Y = ttb.sptensor.from_data(np.array([[0,0],[0,1]]),np.array([[3],[3]]),(2,2))

In [70]:
(X == Y)

Sparse tensor of shape 2 x 2 with 1 nonzeros 
	[1, 0] = True

In [71]:
X.isequal(Y)

False

In [72]:
X != Y

Sparse tensor of shape 2 x 2 with 3 nonzeros 
	[1, 1] = True
	[0, 1] = True
	[0, 0] = True

## Use permute to reorder the modes of a Sptensor

In [73]:
np.random.seed(0)
X = ttb.sptenrand((30, 40, 20, 1), nonzeros = 5) # Create data.
X

Sparse tensor of shape 30 x 40 x 20 x 1 with 5 nonzeros 
	[0, 33, 15, 0] = 0.978618342232764
	[12, 25, 8, 0] = 0.7991585642167236
	[16, 28, 12, 0] = 0.46147936225293185
	[17, 37, 1, 0] = 0.7805291762864555
	[28, 15, 15, 0] = 0.11827442586893322

In [74]:
X.permute(np.array([3, 2, 1, 0])) # Reorder the modes. 

Sparse tensor of shape 1 x 20 x 40 x 30 with 5 nonzeros 
	[0, 15, 33, 0] = 0.978618342232764
	[0, 8, 25, 12] = 0.7991585642167236
	[0, 12, 28, 16] = 0.46147936225293185
	[0, 1, 37, 17] = 0.7805291762864555
	[0, 15, 15, 28] = 0.11827442586893322

Permute works correctly for a 1-dimensional sptensor.

In [75]:
np.random.seed(0)
X = ttb.sptenrand((40,), nonzeros = 4) # Create data.
X

Sparse tensor of shape 40 with 4 nonzeros 
	[16] = 0.9636627605010293
	[17] = 0.3834415188257777
	[25] = 0.7917250380826646
	[35] = 0.5288949197529045

In [76]:
X.permute(np.array([0]))

Sparse tensor of shape 40 with 4 nonzeros 
	[16] = 0.9636627605010293
	[17] = 0.3834415188257777
	[25] = 0.7917250380826646
	[35] = 0.5288949197529045

## Displaying a Sptensor

In [77]:
print(X)

Sparse tensor of shape 40 with 4 nonzeros 
	[16] = 0.9636627605010293
	[17] = 0.3834415188257777
	[25] = 0.7917250380826646
	[35] = 0.5288949197529045


In [78]:
X # In the python interface

Sparse tensor of shape 40 with 4 nonzeros 
	[16] = 0.9636627605010293
	[17] = 0.3834415188257777
	[25] = 0.7917250380826646
	[35] = 0.5288949197529045